In [1]:
import os,sys
import pandas as pd
import numpy as np
sys.path.append(os.path.normpath(os.getcwd()))

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
metadata = pd.read_csv('E:/recomendar/Project/data/metadata_prep.csv')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45068, 75551)

In [11]:
#from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#cosine_sim = linear_kernal(tfidf_matrix, tfidf_matrix)

In [12]:
def get_recommendations(title, df, indices, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 3 most similar movies
    sim_scores = sim_scores[1:4]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 3 most similar movies
    return df['title'].iloc[movie_indices]

In [13]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [23]:
%%timeit
get_recommendations('Balto', metadata, indices, cosine_sim)

21 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
a = get_recommendations('Balto', metadata, indices, cosine_sim)
a

18615     The Dark Side of the Sun
33832          Three Little Wolves
19968    Spring Break Shark Attack
Name: title, dtype: object

In [20]:
metadata.loc[a.index]['title']

18615     The Dark Side of the Sun
33832          Three Little Wolves
19968    Spring Break Shark Attack
Name: title, dtype: object

In [21]:
metadata.loc[a.index]['imdbURL']

18615    https://www.imdb.com/title/tt0118930/
33832    https://www.imdb.com/title/tt0028368/
19968    https://www.imdb.com/title/tt0450326/
Name: imdbURL, dtype: object

In [24]:
metadata.loc[a.index]['overview']

18615          He risks it all for the love of a lifetime!
33832    Two little pigs cry wolf on their brother and ...
19968    A young teenager travels to Florida, unknowing...
Name: overview, dtype: object